In [1]:
import pandas as pd
import datetime as dt

In [2]:
games = pd.read_csv(r"C:\Users\naren\Downloads\games.csv")
games.drop(['Unnamed: 0'], axis=1, inplace = True)
games.dropna(inplace = True)
games.drop_duplicates(inplace = True)
games.reset_index(inplace = True, drop = True)
for i in range(len(games)):
	if games['Release Date'][i]=='releases on TBD':
		games['Release Date'][i]='Dec 31, 2023'

format_1 = '%b %d, %Y'
for i in range(len(games)):
	games['Release Date'][i] = dt.datetime.strptime(games['Release Date'][i], format_1)

games['Release Date'] = pd.to_datetime(games['Release Date'])
print("length: " + str(len(games)))
print(games.head(100))

length: 1116
                                      Title Release Date  \
0                                Elden Ring   2022-02-25   
1                                     Hades   2019-12-10   
2   The Legend of Zelda: Breath of the Wild   2017-03-03   
3                                 Undertale   2015-09-15   
4                             Hollow Knight   2017-02-24   
..                                      ...          ...   
95                 Metroid Prime Remastered   2023-02-08   
96                            Halo Infinite   2021-11-15   
97                    Batman: Arkham Asylum   2009-08-25   
98                     Super Mario Sunshine   2002-07-19   
99                                 Mother 3   2006-04-20   

                                                 Team  Rating Times Listed  \
0      ['Bandai Namco Entertainment', 'FromSoftware']     4.5         3.9K   
1                                ['Supergiant Games']     4.3         2.9K   
2   ['Nintendo', 'Nintendo EPD P

C:\Users\naren\AppData\Local\Temp\ipykernel_22312\3081101913.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  games['Release Date'][i]='Dec 31, 2023'
C:\Users\naren\AppData\Local\Temp\ipykernel_22312\3081101913.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  games['Release Date'][i] = dt.datetime.strptime(games['Release Date'][i], format_1)


In [3]:
ls = []
for i in range(len(games)):
 for q in range(len(games['Reviews'][i].split())):
  ls.append(games['Reviews'][i].split()[q])

ls= [x for x in ls if x.isalpha()]
ls = list(set(ls))
ls = [x for x in ls if len([q for q in games['Reviews'] if x in q.split()])>100]
pd.DataFrame(ls).head(100)

,0
0,em
1,any
2,would
3,ever
4,after
...,...
95,well
96,hours
97,made
98,think


In [4]:
ls1 = []
for i in range(len(ls)):
	ls1.append(games[[ls[i] in c for c in list(games['Reviews'])]]['Rating'].mean())

ls2 = sorted(ls1, reverse = True)

ls3 = []
for i in range(10):
 ls3.append(ls[ls1.index(ls2[i])])

print(ls3)
print([round(ls2[q], 4) for q in [i for i in range(10)]])

['favorite', 'characters', 'best', 'will', 'whole', 'different', 'love', 'character', 'music', 'world']
[3.8772, 3.8155, 3.811, 3.7913, 3.7808, 3.7804, 3.7759, 3.759, 3.7581, 3.7548]


In [5]:
ls = []
for i in range(len(games)):
 for q in range(len(games['Summary'][i].split())):
  ls.append(games['Summary'][i].split()[q])

ls= [x for x in ls if x.isalpha()]
ls = list(set(ls))
ls = [x for x in ls if len([q for q in games['Summary'] if x in q.split()])>100]
pd.DataFrame(ls).head(100)

,0
0,features
1,against
2,new
3,this
4,for
...,...
61,more
62,they
63,other
64,action


In [6]:
ls1 = []
for i in range(len(ls)):
	ls1.append(games[[ls[i] in c for c in list(games['Reviews'])]]['Rating'].mean())

ls4 = sorted(ls1, reverse = True)

ls5 = []
for i in range(10):
 ls5.append(ls[ls1.index(ls4[i])])

print(ls5)
print([round(ls4[q], 4) for q in [i for i in range(10)]])

['must', 'characters', 'will', 'world', 'story', 'who', 'through', 'time', 'action', 'into']
[3.8245, 3.8155, 3.7913, 3.7548, 3.7411, 3.7286, 3.728, 3.7056, 3.7031, 3.7011]


In [7]:
class getwords:
	def __init__(self, startyear,endyear): 
		gamelist = games[(games['Release Date'].dt.year>=startyear) & (games['Release Date'].dt.year<=endyear)]
		self.gamelist = gamelist.reset_index(drop = True)
		if len(self.gamelist)==0:
			raise Exception("No games in selection.")
	
	def reviewwords(self):
		ls = []
		for i in range(len(self.gamelist)):
			for q in range(len(self.gamelist['Reviews'][i].split())):
				ls.append(self.gamelist['Reviews'][i].split()[q])
		ls= [x for x in ls if x.isalpha()]
		ls = list(set(ls))
		ls = [x for x in ls if len([q for q in self.gamelist['Reviews'] if x in q.split()])>10]
		if len(ls)<=10:
			raise Exception("Selection too short.")
		ls1 = []
		for i in range(len(ls)):
			ls1.append(self.gamelist[[ls[i] in c for c in list(self.gamelist['Reviews'])]]['Rating'].mean())
		self.ratings = sorted(ls1, reverse = True)
		self.words = []
		for i in range(10):
			self.words.append(ls[ls1.index(self.ratings[i])])
		self.topwords = {}
		for i in range(10):
			self.topwords.update({self.words[i]:round(self.ratings[i], 4)})
		return self.topwords
	
	def summarywords(self):
		ls = []
		for i in range(len(self.gamelist)):
			for q in range(len(self.gamelist['Summary'][i].split())):
				ls.append(self.gamelist['Summary'][i].split()[q])
		ls= [x for x in ls if x.isalpha()]
		ls = list(set(ls))
		ls = [x for x in ls if len([q for q in self.gamelist['Summary'] if x in q.split()])>10]
		if len(ls)<=10:
			raise Exception("Selection too short.")
		ls1 = []
		for i in range(len(ls)):
			ls1.append(self.gamelist[[ls[i] in c for c in list(self.gamelist['Summary'])]]['Rating'].mean())
		self.ratings = sorted(ls1, reverse = True)
		self.words = []
		for i in range(10):
			self.words.append(ls[ls1.index(self.ratings[i])])
		self.topwords = {}
		for i in range(10):
			self.topwords.update({self.words[i]:round(self.ratings[i], 4)})
		return self.topwords
    


reviewwordlist = getwords(2015,2022).reviewwords()
summarywordlist = getwords(2015,2022).summarywords()
print("Words in the reviews of the most popular games and their average ratings: \n"+str(reviewwordlist)+"\n")
print("Words in the summaries of the most popular games and their average ratings: \n"+str(summarywordlist)+"\n")

Words in the reviews of the most popular games and their average ratings: 
{'themes': 4.0333, 'stories': 4.0208, 'finally': 4.0095, 'incredible': 3.9857, 'sometimes': 3.984, 'emotional': 3.9786, 'attention': 3.9769, 'masterpiece': 3.9667, 'ability': 3.95, 'replay': 3.9469}

Words in the summaries of the most popular games and their average ratings: 
{'solve': 4.0647, 'includes': 4.0176, 'such': 3.9769, 'called': 3.9636, 'RPG': 3.9239, 'tale': 3.9059, 'remastered': 3.9, 'true': 3.9, 'fully': 3.8962}



In [8]:
class populargames(getwords):
	def __init__(self, startyear,endyear):
		self.reviewwords= getwords(startyear,endyear).reviewwords()
		self.summarywords = getwords(startyear,endyear).summarywords()
		self.gamelist = getwords(startyear,endyear).gamelist
	def findreviewwords(self,position):
		if not 0<position<11:
			raise Exception("Out of range.")
		print("\n" + "Games with reviews that contain the word "+str(list(self.reviewwords.keys())[position-1]) + ".\n")
		print(self.gamelist[self.gamelist['Reviews'].str.contains(str(list(self.reviewwords.keys())[position-1]))]['Title'])
	def findsummarywords(self,position):
		if not 0<position<11:
			raise Exception("Out of range.")
		print("\n" + "Games with summaries that contain the word "+str(list(self.summarywords.keys())[position-1]) + ".\n")
		print(self.gamelist[self.gamelist['Summary'].str.contains(str(list(self.summarywords.keys())[position-1]))]['Title'])



In [9]:
populargames(2015,2022).findreviewwords(1)


Games with reviews that contain the word themes.

22                         Disco Elysium: The Final Cut
37                               The Last of Us Part II
68                                              Control
84                                        Disco Elysium
93                                      Pokémon Scarlet
107               The Legend of Zelda: Link's Awakening
169                                                Soma
174                                               Norco
177                          Uncharted: The Lost Legacy
189                            Chicory: A Colorful Tale
190                      Outer Wilds: Echoes of the Eye
254                Donkey Kong Country: Tropical Freeze
331                      The 25th Ward: The Silver Case
345                   The Great Ace Attorney 2: Resolve
379    Danganronpa Another Episode: Ultra Despair Girls
Name: Title, dtype: object


In [10]:
populargames(2015,2022).findsummarywords(1)


Games with summaries that contain the word solve.

30                                             Signalis
38                             The Witcher 3: Wild Hunt
89                                 Deltarune: Chapter 1
99           Stranger of Paradise: Final Fantasy Origin
110          AI: The Somnium Files - Nirvana Initiative
143                    Hollow Knight: Voidheart Edition
167                          Uncharted 4: A Thief's End
189                            Chicory: A Colorful Tale
219                                                Pyre
266                            The Witcher 3: Wild Hunt
312                                     The Silver Case
321                                  The Forgotten City
345                   The Great Ace Attorney 2: Resolve
346                                             Bugsnax
384                                            Darkwood
400                 Bug Fables: The Everlasting Sapling
450    Mario & Luigi: Superstar Saga + Bowser's Mini